In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split


import pyTsetlinMachine
from pyTsetlinMachine.tm import MultiClassTsetlinMachine
from pyTsetlinMachine.tm import RegressionTsetlinMachine
from pyTsetlinMachine.tools import Binarizer


ModuleNotFoundError: No module named 'pyTsetlinMachine'

In [ ]:
number_of_folds = 24

In [ ]:
def k10Fold(nfolds,data):
    isOdd = False
    efsize = len(data)//nfolds
    if len(data)-efsize*nfolds:
        isOdd = True
    folds = []
    foldscounter = 0
    counter = 0
    curfold = []
    i = 0
    for item in data:
        if foldscounter < nfolds+1:   #rows / folds
            if counter < efsize :
                curfold.append(i)
                counter += 1
            else:
                foldscounter+=1
                if foldscounter != nfolds:
                    folds.append(curfold)
                    
                    curfold = []
                    curfold.append(i)
                    counter = 1
            i+=1
        else:
            curfold.append(i)
            i+=1
    folds.append(curfold)
    print(folds)
    return folds
def trainTestSpliter(testFolIndex,data):
    test_data = []
    train_data = []
    for i in range(len(data)):
        if i not in testFolIndex:
            train_data.append(data[i])
        else:
            test_data.append(data[i])
    return np.array(train_data), np.array(test_data)
def dataTargetSpluter(train_d,test_d):
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    for i in train_d:
        train_x.append(i[:9])
        train_y.append(i[9])
    for i in test_d:
        test_x.append(i[:9])
        test_y.append(i[9])
    return np.array(train_x), np.array(train_y), np.array(test_x), np.array(test_y)

In [ ]:
x = np.loadtxt("deshar.txt")
df = pd.DataFrame(x,columns = ["F1","F2","F3","F4","F5","F6","F7","F8","F9","Effort"])

In [ ]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

In [ ]:
foldsIndex = k10Fold(number_of_folds,scaled_data)

In [ ]:
avg_ame=0
for i in range(number_of_folds):
    print(" ")
    print("Fold ","#",i+1)
    start  = time.time()
    model = RegressionTsetlinMachine(3000, 1400, 3.2)
    
    train_data, test_data = trainTestSpliter(foldsIndex[i],scaled_data)
    train_x, train_y, test_x, test_y = dataTargetSpluter(train_data,test_data)
    
    b = Binarizer(max_bits_per_feature = 10)
    b.fit(train_x)
    X_transformed_train = b.transform(train_x)
    
    b = Binarizer(max_bits_per_feature = 10)
    b.fit(test_x)
    X_transformed_test = b.transform(test_x)
    
    model.fit(X_transformed_train, train_y, epochs=150)
    
    all_data_x =[]
    all_data_y = []

    for i in scaled_data:
        all_data_x.append(i[:9])
        all_data_y.append(i[9])

    all_data_x = np.array(all_data_x)
    all_data_y = np.array(all_data_y)
    
    b = Binarizer(max_bits_per_feature = 10)
    b.fit(all_data_x)
    X_transformed_all_data_x = b.transform(all_data_x)
    prediction = model.predict(X_transformed_all_data_x)
    end = time.time()

    plt.plot(prediction, label="Model Prediction")
    plt.plot(all_data_y, label="Real")
    plt.legend()
    plt.show()


    for i in range(len(prediction)):
        print("Prediction : ", prediction[i],"     ","real :", all_data_y[i])
    print("All-data-MAE: ",mean_absolute_error(all_data_y,prediction))
    
    prediction = model.predict(X_transformed_test)
    print("Test-MAE: ",mean_absolute_error(test_y,prediction))
    avg_ame+=mean_absolute_error(test_y,prediction)
    print("Took: ",end-start, "Seconds")
    print("#################################################################################################")
print("\n \n avg test-MAE : ",avg_ame/number_of_folds)